Author: **Paul F**

# 4. Data visualisation and peak fitting

### Goals

- create your first own Git repository
- reading iSCAMS data from a file and rescale the values
- plot a histogram using the package *matplotlib* and save the plot to a file
- fit a gaussian function to the histogram with the package *scipy* and determine the average protein mass

### Introduction

In this folder you will find the file ``hsp165.txt`` that contains the interferometric contrasts of single proteins (HSP 16.5) measured by iSCAMS (stands for interferometric scattering mass spectrometry). The interferometric contrast is proportional to the particle mass (for this measurement an interferometric contrast of $\approx2.2\times10^{-5}$ corresponds to a protein mass of 1 kDa.

### *TASK 1*

Load the interferometric contrasts from the file into a *numpy-array* of floating point values. Determine the number of values in the array and confirm that the number matches the number of lines in the file ``hsp164.txt`` (check this number for example by opening the file in a text editor).

In [2]:
# Before we start we need to import the maths library
import numpy
# a library with tools for scientific data analysis
import scipy

#entire_pdb = open('1PRE.pdb', 'r')
#pdb_in_lines = entire_pdb.readlines() # List of Strings (each line one element)

infcontr = [] # will be used as list of strings
with open('hsp165.txt', 'r') as file:
    for line in file:
        contr = float(line)
        infcontr.append(contr) 
print(infcontr)

[0.009795546, 0.007485968, 0.008427611, 0.009003952, 0.010191283, 0.008606434, 0.009018369, 0.009149075, 0.007491902, 0.009066961, 0.007227701, 0.009162959, 0.008139796, 0.008438112, 0.007763638, 0.007257782, 0.007284599, 0.006689628, 0.007433876, 0.00761264, 0.009261033, 0.009260974, 0.007781852, 0.007598163, 0.008704568, 0.009871311, 0.007446929, 0.008721299, 0.011159862, 0.009709397, 0.00812526, 0.008127158, 0.009565282, 0.009812811, 0.010753267, 0.008665053, 0.00800814, 0.009211136, 0.010044083, 0.007564463, 0.008701898, 0.009877779, 0.009562672, 0.007422188, 0.00827513, 0.007742338, 0.00832829, 0.008092687, 0.006787465, 0.009427931, 0.016131383, 0.008616461, 0.014128191, 0.008941892, 0.008708009, 0.007867526, 0.008091797, 0.01111655, 0.008899529, 0.007922941, 0.00725707, 0.009840637, 0.010927284, 0.014709457, 0.008298744, 0.008763602, 0.007907693, 0.008622869, 0.014213628, 0.00750264, 0.00815985, 0.018043977, 0.007858033, 0.007395785, 0.009130623, 0.008753041, 0.006767352, 0.00998

### *TASK 2*

Convert the measured interferometric contrasts into the unit kDa and and use the *numpy-array* methods to determine the 

- minimum
- maximum
- mean
- standard deviation

of the converted values and print your results to the screen.

In [71]:
# TYPE YOUR SOLUTION HERE

### *TASK 3*

a) Use the function [*matplotlib.pyplot.histogram*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to calculate and display the histogram of the measured values.

b) Adjust the parameters ``bins`` and ``range`` to improve your plot.


c) Annotate your axes by using the functions [*matplotlib.pyplot.xlabel*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.xlabel.html) and [*matplotlib.pyplot.ylabel*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.ylabel.html).

d) Save the histogram to a PNG file ``hist_hsp.png`` at 400 dpi resolution (search on the web for the suitable *matplotlib* function).

In [40]:
# Import of the matplotlib package for plotting
import matplotlib
# and magic to activate inline-plotting between the notebook cells
%matplotlib inline

In [ ]:
# TYPE YOUR SOLUTION HERE

### *TASK 4*

a) The function [*matplotlib.pyplot.histogram*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) returns three objects. We are interested in the first two:

  - *numpy-array* of histogram values (number of particles that had a mass within the limits of the respective bin)
  - *numpy-array* of the limits of the histogram bins

 Save the two arrays as ``hist`` and ``m_edges`` and check their lengths. Why are they not the same?
 Save the third array into the variable ``_``.
 
b) Calculate the *numpy-array* ``m_centers`` with values between the values of ``m_edges`` and a length ``len(m_edges)-1``.

c) Make a line plot of ``m_centers`` vs. ``hist`` by using the function [*matplotlib.pyplot.plot*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html). 

d) Label the axes.

e) Use the same function as in *TASK 3* to save the plot to the PNG file ``hist_hsp_line.png`` at 400 dpi resolution.

In [23]:
# TYPE YOUR SOLUTION HERE

### *TASK 4*

In taks 5 we want to determine the mass of HSP 16.5 by fitting a Gaussian function to the histogram.

The Gaussian function is defined as 

$$g(x) = A_0 \exp\left( -\frac{\left(m-m_0\right)^2}{2 \sigma^2} \right)$$

a) Write a corresponding Python function ``gauss(m, A0, m0, sigma)`` that returns $g(x)$.

b) Assign the default values $A_0=1$, $m_0=0$, $\sigma=1$.

c) Confirm that the function returns the value $e^{-1/2}\approx0.61$ for $m=1$, $A_0=1$, $m_0=0$, and $\sigma=1$. 

In [72]:
# TYPE YOUR SOLUTION HERE

### *TASK 5*

The function [scipy.optimize.least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) can solve nonlinear least-squares problems. We will use this function to identify the Gaussian model (with parameters $A_0$, $m_0$, and $\sigma$) that best matches the measured data. The fitted values $m_0$ and $\sigma$ provide estimates for the mass of HSP 16.5 and the spread of the measured values, respectively.

a) Define a function ``err(x)`` that calculates the fit residual (difference of fit and measured data). For using this function later to call [scipy.optimize.least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html) the argument of ``err`` has to be a tuple of all model parameters, here ``x=(A0, x0, sigma)``.

b) For convergence to the correct solution a guess for the model parameters has to be passed to [scipy.optimize.least_squares](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html). Define a starting guess ``x0`` as a tuple of three guessed values for $A_0$, $m_0$, and $\sigma$. Use the plot that you generated in *TASK 4* to guess the parameters.

c) Call ``(A0_fit, x0_fit, sigma_fit) = scipy.optimize.least_squares(err, x0).x0`` and both data and result with consecutive calls of [*matplotlib.pyplot.plot*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html).

d) Print the fitted parameters $m_0$ and $\sigma$ to the screen and compare to the values obtained in *TASK 2*.

**Advanced**: Make a legend by using the ``label`` parameter of [*matplotlib.pyplot.plot*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) and by calling at the end of the cell [*matplotlib.pyplot.legend*](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.legend.html).

In [78]:
# TYPE YOUR SOLUTION HERE